In [660]:
%matplotlib inline
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from scipy import stats
from ast import literal_eval # 문자열 모형의 딕트를 스근하게 딕트로 바꾸어 준다. 
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from surprise import Reader, Dataset, SVD, evaluate


In [661]:
dataSet = pd.read_csv('crawler/top.csv')
dataSet = dataSet.drop(columns='Unnamed: 0', axis=1)
dataSet2 = dataSet.copy()

In [662]:
name_popId = dataSet[['name', 'cosmeticId']]

In [663]:
name_popId = name_popId.drop_duplicates().set_index('name')

In [664]:
dataSet

,name,cosmeticId,rate,review,type
0,비타민 워터리 선 젤 SPF50+ PA++++,0,4.33,일본 비오레 아쿠아 썬크림 다 써서 최대한 비슷한거 찾아봤더니 이 제품이 제일 비슷...,건성
1,비타민 워터리 선 젤 SPF50+ PA++++,0,4.33,글픽보고 세일때 샀는데 끈적임없고 흡수가 빨랐어요!! 향도 레몬?비타민?같은 향이라...,건성
2,비타민 워터리 선 젤 SPF50+ PA++++,0,4.33,우오 나 이거 3위일때인가 샀는데 지금 1위네영?\r\n가지고 다니던 썬크림을 잃어...,건성
3,비타민 워터리 선 젤 SPF50+ PA++++,0,4.33,일단 제형이 수분감 있어서 좋구 백탁없구 쓰기 너무 좋아요 하지만..저에겐 눈시림이...,건성
4,비타민 워터리 선 젤 SPF50+ PA++++,0,4.50,끈적임없고 세미매트한 마무리감 톤업이 되지않는 제품이라서 팔에 바르기가 좋고 메이크...,지성
5,비타민 워터리 선 젤 SPF50+ PA++++,0,4.50,남자친구도 잘 쓰고 있는 선크림이에요! 자극성도 없구! 예민한 피부에 딱이에요 ??,지성
6,비타민 워터리 선 젤 SPF50+ PA++++,0,4.50,식물나라 선젤이랑 비슷하다는 후기보고\r\n바로 올영가서 테스트해봄\r\n\r\n식...,지성
7,비타민 워터리 선 젤 SPF50+ PA++++,0,4.50,올리브영에서 홍보 많이하길래 사봤어요\r\n기존에 선크림 정착템은 식물나라 산소수선...,지성
8,비타민 워터리 선 젤 SPF50+ PA++++,0,3.88,산뜻하게발리고 차단지수도 높아서 나쁘지않아요!,중성
9,비타민 워터리 선 젤 SPF50+ PA++++,0,3.88,"- 선크림으로도, 메이크업 전 베이스로도 기능을 잘 하는 아이템\r\n\r\n- 원...",중성


In [665]:
dataSet = dataSet.drop('name', axis=1)

In [666]:
dataSet.head(5)

,cosmeticId,rate,review,type
0,0,4.33,일본 비오레 아쿠아 썬크림 다 써서 최대한 비슷한거 찾아봤더니 이 제품이 제일 비슷...,건성
1,0,4.33,글픽보고 세일때 샀는데 끈적임없고 흡수가 빨랐어요!! 향도 레몬?비타민?같은 향이라...,건성
2,0,4.33,우오 나 이거 3위일때인가 샀는데 지금 1위네영?\r\n가지고 다니던 썬크림을 잃어...,건성
3,0,4.33,일단 제형이 수분감 있어서 좋구 백탁없구 쓰기 너무 좋아요 하지만..저에겐 눈시림이...,건성
4,0,4.50,끈적임없고 세미매트한 마무리감 톤업이 되지않는 제품이라서 팔에 바르기가 좋고 메이크...,지성


In [667]:
tf = TfidfVectorizer(analyzer='word', min_df=2, stop_words=['\r', '\n'], sublinear_tf=True)
tf_matrix = tf.fit_transform(dataSet['review'])

In [668]:
tf_matrix

<1923x5156 sparse matrix of type '<class 'numpy.float64'>'
	with 40215 stored elements in Compressed Sparse Row format>

In [669]:
cosine_sim = linear_kernel(tf_matrix, tf_matrix)
cosine_sim[0]
reader = Reader() 

In [670]:
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1923 entries, 0 to 1922
Data columns (total 4 columns):
cosmeticId    1923 non-null int64
rate          1923 non-null float64
review        1923 non-null object
type          1923 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 60.2+ KB


In [671]:
type = {'건성': 0, '지성': 1, '중성': 2, '복합성': 3, '민감성': 4}
dataSet['type'] = dataSet['type'].map(type)
data = Dataset.load_from_df(dataSet[['cosmeticId', 'type', 'rate']] , reader)

In [672]:
data.split(n_folds=5)

In [673]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1


C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


RMSE: 0.1808
MAE:  0.1356
------------
Fold 2


RMSE: 0.1821
MAE:  0.1344
------------
Fold 3


RMSE: 0.2243
MAE:  0.1417
------------
Fold 4


RMSE: 0.1730
MAE:  0.1289
------------
Fold 5


RMSE: 0.1975
MAE:  0.1426
------------
------------
Mean RMSE: 0.1915
Mean MAE : 0.1366
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.18075649979371883,
                             0.18214220380306156,
                             0.22431900072910682,
                             0.17299200425989747,
                             0.19748680410994626],
                            'mae': [0.13558760793995497,
                             0.13436088101846042,
                             0.14165759941095202,
                             0.12892485499073833,
                             0.14260980549624916]})

In [674]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [712]:
def type_cosmeticsNum(type, cosmeticName=''):
    if cosmeticName != '':
        idx = name_popId['cosmeticId'][cosmeticName]
        print(idx)
    else:
        sortRate = dataSet[dataSet['type'] == type].drop_duplicates(['cosmeticId'])
        sortRate = sortRate.sort_values('rate', ascending=False)
        idx = sortRate['cosmeticId'][:10]
        sortRate = sortRate.reset_index(drop=True)
        sortRate['lenReview'] = 0
        for i, k in enumerate(idx):
            sortRate.loc[i, 'lenReview'] = len(dataSet2[dataSet2['cosmeticId'] == k]['review'])
        sortRate = sortRate.sort_values('lenReview', ascending=False)
        idx = sortRate['cosmeticId'][:1]
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    cosmetic_indcies = [i[0] for i in sim_scores]
    cosmetics = dataSet.iloc[cosmetic_indcies][['cosmeticId', 'rate', 'review', 'type']]
    cosmetics = cosmetics[cosmetics['type'] == type]
    cosmetics['est'] = cosmetics['cosmeticId'].apply(lambda x:
                                                     svd.predict(x, type).est)
    cosmetics = cosmetics.sort_values('est', ascending=False)
    finalCosmetics = cosmetics.drop_duplicates(['cosmeticId'])
    finalCosmetics = finalCosmetics['cosmeticId'].reset_index(drop=True)
    preCosmeticName = name_popId.index[finalCosmetics]
    for name in preCosmeticName[:10]:
        print(name)


In [ ]:
type_cosmeticsNum(4, '퍼펙트 UV 프로텍터 (파란자차) SPF50+ PA+++')

9
파워블록 에센스 선크림 SPF50+PA++++
UV 아쿠아 리치 워터리 젤 SPF 50+ PA++++
에브리 선데이
마일드 프로텍트 에센스 선 SPF50+ PA++++
오트 마일드 모이스처 선크림 SPF50+ PA+++
선 메이트 프로텍터 SPF50+ PA+++


In [677]:
dataSet

,cosmeticId,rate,review,type
0,0,4.33,일본 비오레 아쿠아 썬크림 다 써서 최대한 비슷한거 찾아봤더니 이 제품이 제일 비슷...,0
1,0,4.33,글픽보고 세일때 샀는데 끈적임없고 흡수가 빨랐어요!! 향도 레몬?비타민?같은 향이라...,0
2,0,4.33,우오 나 이거 3위일때인가 샀는데 지금 1위네영?\r\n가지고 다니던 썬크림을 잃어...,0
3,0,4.33,일단 제형이 수분감 있어서 좋구 백탁없구 쓰기 너무 좋아요 하지만..저에겐 눈시림이...,0
4,0,4.50,끈적임없고 세미매트한 마무리감 톤업이 되지않는 제품이라서 팔에 바르기가 좋고 메이크...,1
5,0,4.50,남자친구도 잘 쓰고 있는 선크림이에요! 자극성도 없구! 예민한 피부에 딱이에요 ??,1
6,0,4.50,식물나라 선젤이랑 비슷하다는 후기보고\r\n바로 올영가서 테스트해봄\r\n\r\n식...,1
7,0,4.50,올리브영에서 홍보 많이하길래 사봤어요\r\n기존에 선크림 정착템은 식물나라 산소수선...,1
8,0,3.88,산뜻하게발리고 차단지수도 높아서 나쁘지않아요!,2
9,0,3.88,"- 선크림으로도, 메이크업 전 베이스로도 기능을 잘 하는 아이템\r\n\r\n- 원...",2


In [678]:
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1923 entries, 0 to 1922
Data columns (total 4 columns):
cosmeticId    1923 non-null int64
rate          1923 non-null float64
review        1923 non-null object
type          1923 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 60.2+ KB


In [679]:
sortRate = dataSet[dataSet['type']==1].drop_duplicates(['cosmeticId'])
sortRate = sortRate.sort_values('rate', ascending=False)
sortRate.loc['cosmeticId'][1]

KeyError: 'the label [cosmeticId] is not in the [index]'